In [2]:
import warnings
warnings.simplefilter("ignore")

import time
import logging
import numpy as np
import os 
import sys
import yaml
import pyomo.environ as pyomo_env
import pandas as pd
from scipy.spatial import ConvexHull,  Delaunay
from scipy.interpolate import griddata
from multiprocessing import Lock, Process, Queue, current_process
import queue # imported for using queue.Empty exception
sys.path.append(os.getcwd())
import plotly.graph_objects as go
import plotly.figure_factory as ff
warnings.simplefilter("ignore")
import pyomo
import matplotlib.pyplot as plt

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("pypsa_tools.pypsaTools", "/home/tim/Dropbox/Speciale/PyPSA_project/pypsa_tools/pypsaTools.py")
pt = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pt)
spec2 = importlib.util.spec_from_file_location("pypsa_tools.plottingTools", "/home/tim/Dropbox/Speciale/PyPSA_project/pypsa_tools/plottingTools.py")
pt_plot = importlib.util.module_from_spec(spec2)
spec2.loader.exec_module(pt_plot)
spec3 = importlib.util.spec_from_file_location("pypsa_tools.arTools", "/home/tim/Dropbox/Speciale/PyPSA_project/pypsa_tools/arTools.py")
ar = importlib.util.module_from_spec(spec3)
spec3.loader.exec_module(ar)

In [7]:
P= np.array( [[0,0,1,1,0,0,1,1],[0,1,1,0,0,1,1,0],[0,0,0,0,1,1,1,1]]).T

In [67]:
import itertools
n=4
P = np.array(list(itertools.product([1,0], repeat=n)))

In [68]:
fig = go.Figure(go.Scatter3d(x=P[:,0],y=P[:,1],z=P[:,2]))
fig.show()

In [69]:
A,b = ar.vert2con(P)

In [70]:
A

array([[-2., -0., -0., -0.],
       [-2., -0., -0., -0.],
       [-2., -0., -0., -0.],
       [-2., -0., -0.,  0.],
       [-2., -0., -0.,  0.],
       [-2., -0., -0.,  0.],
       [ 0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  2.],
       [ 0., -0.,  0.,  2.],
       [ 0., -0., -0.,  2.],
       [ 0.,  0., -0.,  2.],
       [ 0., -0., -0.,  2.],
       [ 0.,  0.,  2.,  0.],
       [ 0.,  0.,  2.,  0.],
       [ 0., -0.,  2.,  0.],
       [ 0., -0.,  2., -0.],
       [ 0.,  0.,  2.,  0.],
       [ 0., -0.,  2.,  0.],
       [ 0., -2., -0., -0.],
       [ 0., -2., -0., -0.],
       [ 0., -2., -0., -0.],
       [ 0., -2., -0.,  0.],
       [ 0., -2., -0.,  0.],
       [ 0., -2., -0.,  0.],
       [ 0.,  2.,  0.,  0.],
       [ 0.,  2.,  0., -0.],
       [ 0.,  2.,  0.,  0.],
       [ 0.,  2.,  0., -0.],
       [ 0.,  2.,  0.,  0.],
       [ 0.,  2., -0.,  0.],
       [ 2., -0.,  0.,  0.],
       [ 2., -0.,  0.,  0.],
       [ 2., -0.,  0.,  0.],
       [ 2., -0.,  0., -0.],
       [ 2., -

In [71]:
b

array([0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [72]:

def calc_multi_test(point):

    A,b = ar.vert2con(P)
    AA = np.array([A[:,i]-A[:,-1] for i in range(A.shape[1]-1)]).T
    bb = b-A[:,-1]*point


    V = ar.con2vert(AA,bb)
    H = ConvexHull(V).volume
    return H

In [73]:
x =  np.linspace(0.1,2.9,100)
x

array([0.1       , 0.12828283, 0.15656566, 0.18484848, 0.21313131,
       0.24141414, 0.26969697, 0.2979798 , 0.32626263, 0.35454545,
       0.38282828, 0.41111111, 0.43939394, 0.46767677, 0.4959596 ,
       0.52424242, 0.55252525, 0.58080808, 0.60909091, 0.63737374,
       0.66565657, 0.69393939, 0.72222222, 0.75050505, 0.77878788,
       0.80707071, 0.83535354, 0.86363636, 0.89191919, 0.92020202,
       0.94848485, 0.97676768, 1.00505051, 1.03333333, 1.06161616,
       1.08989899, 1.11818182, 1.14646465, 1.17474747, 1.2030303 ,
       1.23131313, 1.25959596, 1.28787879, 1.31616162, 1.34444444,
       1.37272727, 1.4010101 , 1.42929293, 1.45757576, 1.48585859,
       1.51414141, 1.54242424, 1.57070707, 1.5989899 , 1.62727273,
       1.65555556, 1.68383838, 1.71212121, 1.74040404, 1.76868687,
       1.7969697 , 1.82525253, 1.85353535, 1.88181818, 1.91010101,
       1.93838384, 1.96666667, 1.99494949, 2.02323232, 2.05151515,
       2.07979798, 2.10808081, 2.13636364, 2.16464646, 2.19292

In [74]:
y = [calc_multi_test([i]) for i in x]

In [75]:
go.Figure(go.Scatter(x=x,y=y))

In [78]:
y2 = x**2 * np.sqrt(3) / 3

In [79]:
go.Figure(go.Scatter(x=x,y=y2))